In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.layers import LSTM, Dense
from keras.models import Sequential
from glob import glob
import pickle

In [2]:
data_files = glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\time_series_analysis\\final_project\\nasdaq_stock_price_y5_daily\*")
print(len(data_files))


3659


AAPL is traded daily, so it will have the full date index. Will set that as the initial index and then can merge off of that dataframe

In [3]:
aapl_file_path = [a for a in data_files if 'AAPL' in a][0]
data_aapl = pd.read_csv(aapl_file_path)
data_aapl = data_aapl.set_index('Date')

stock_symbol = 'AAPL'
rename_dict = {'Open':stock_symbol + '_Open', 'High':stock_symbol+'_High',
              'Low':stock_symbol+'_Low', 'Close':stock_symbol+'_Close', 
               'Volume':stock_symbol+'_Volume', 'Dividends':stock_symbol+'_Dividends', 
               'Stock Splits':stock_symbol+'_Stock_Splits'}
data_aapl = data_aapl.rename(columns=rename_dict)
data_aapl.head()

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,AAPL_Dividends,AAPL_Stock_Splits
Date,,,,,,,
2019-03-06 00:00:00-05:00,42.038652,42.236007,41.862960,42.002552,83241600,0.0,0.0
2019-03-07 00:00:00-05:00,41.846107,41.983294,41.400861,41.516384,99185600,0.0,0.0
2019-03-08 00:00:00-05:00,40.991724,41.653579,40.794369,41.615070,95997600,0.0,0.0
2019-03-11 00:00:00-04:00,42.236007,43.109654,42.202313,43.056705,128044000,0.0,0.0
2019-03-12 00:00:00-04:00,43.321448,43.964049,43.169821,43.540462,129870400,0.0,0.0


In [4]:
print(len(data_aapl))

1260


Full dataframe is over 100mb, so can't upload to github. To aggregate data to a dataframe, can search for the files in the folder with this code and search to stock symbol.

aapl_file_path = [a for a in data_files if 'AAPL' in a][0]


In [5]:
### Merging all of the data into one dataframe

data = data_aapl.copy()
for file in data_files:
    if file != aapl_file_path:
        
        data_one_stock = pd.read_csv(file)
        data_one_stock = data_one_stock.set_index('Date')
        
        stock_symbol = file.split('\\')[-1].split('.csv')[0].split('-')[0]
        
        rename_dict = {}
        for col in data_one_stock.columns:
            rename_dict[col] = stock_symbol+'_' + col
    
        data_one_stock = data_one_stock.rename(columns=rename_dict)
    
    
        data = data.merge(data_one_stock, on='Date', how='outer')

In [6]:
data

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,AAPL_Dividends,AAPL_Stock_Splits,A_Open,A_High,A_Low,...,ZVSA_Volume,ZVSA_Dividends,ZVSA_Stock Splits,ZWS_Open,ZWS_High,ZWS_Low,ZWS_Close,ZWS_Volume,ZWS_Dividends,ZWS_Stock Splits
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-06 00:00:00-05:00,42.038652,42.236007,41.862960,42.002552,83241600,0.0,0.0,77.015197,77.169578,75.963502,...,NaN,NaN,NaN,25.870498,26.063275,25.513863,25.513863,460600,0.0,0.0
2019-03-07 00:00:00-05:00,41.846107,41.983294,41.400861,41.516384,99185600,0.0,0.0,75.924890,76.320479,75.268785,...,NaN,NaN,NaN,25.513856,25.523495,24.983723,25.321081,380100,0.0,0.0
2019-03-08 00:00:00-05:00,40.991724,41.653579,40.794369,41.615070,95997600,0.0,0.0,75.432814,75.490704,74.680224,...,NaN,NaN,NaN,24.964447,25.340360,24.945169,25.186138,343800,0.0,0.0
2019-03-11 00:00:00-04:00,42.236007,43.109654,42.202313,43.056705,128044000,0.0,0.0,75.529311,76.677498,75.510017,...,NaN,NaN,NaN,25.176502,25.957243,24.974087,25.937965,546300,0.0,0.0
2019-03-12 00:00:00-04:00,43.321448,43.964049,43.169821,43.540462,129870400,0.0,0.0,76.783638,77.314307,76.378398,...,NaN,NaN,NaN,25.986160,26.121104,25.774107,25.957245,365400,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 00:00:00-05:00,181.270004,182.570007,179.529999,180.750000,136682600,0.0,0.0,137.509995,138.029999,135.860001,...,3995800.0,0.0,0.0,31.510000,31.820000,31.280001,31.750000,754200,0.0,0.0
2024-03-01 00:00:00-05:00,179.550003,180.529999,177.380005,179.660004,73488000,0.0,0.0,137.029999,139.149994,134.669998,...,1341800.0,0.0,0.0,31.709999,32.150002,31.650000,32.070000,536300,0.0,0.0
2024-03-04 00:00:00-05:00,176.149994,176.899994,173.789993,175.100006,81510100,0.0,0.0,139.149994,143.490005,138.809998,...,1096700.0,0.0,0.0,32.040001,32.509998,32.029999,32.060001,681100,0.0,0.0


Now need to forward fill. The problem is there are some stocks that start with NaN. Could probably remove those.

In [7]:
data = data.fillna(method='ffill')

C:\Users\lvden\AppData\Local\Temp\ipykernel_3140\917065080.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
C:\Users\lvden\AppData\Local\Temp\ipykernel_3140\917065080.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(method='ffill')


In [8]:
data = data.dropna(axis=1)

In [9]:
data

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,AAPL_Dividends,AAPL_Stock_Splits,A_Open,A_High,A_Low,...,ZUO_Volume,ZUO_Dividends,ZUO_Stock Splits,ZWS_Open,ZWS_High,ZWS_Low,ZWS_Close,ZWS_Volume,ZWS_Dividends,ZWS_Stock Splits
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-06 00:00:00-05:00,42.038652,42.236007,41.862960,42.002552,83241600,0.0,0.0,77.015197,77.169578,75.963502,...,957400,0.0,0.0,25.870498,26.063275,25.513863,25.513863,460600,0.0,0.0
2019-03-07 00:00:00-05:00,41.846107,41.983294,41.400861,41.516384,99185600,0.0,0.0,75.924890,76.320479,75.268785,...,1130400,0.0,0.0,25.513856,25.523495,24.983723,25.321081,380100,0.0,0.0
2019-03-08 00:00:00-05:00,40.991724,41.653579,40.794369,41.615070,95997600,0.0,0.0,75.432814,75.490704,74.680224,...,1086800,0.0,0.0,24.964447,25.340360,24.945169,25.186138,343800,0.0,0.0
2019-03-11 00:00:00-04:00,42.236007,43.109654,42.202313,43.056705,128044000,0.0,0.0,75.529311,76.677498,75.510017,...,1540600,0.0,0.0,25.176502,25.957243,24.974087,25.937965,546300,0.0,0.0
2019-03-12 00:00:00-04:00,43.321448,43.964049,43.169821,43.540462,129870400,0.0,0.0,76.783638,77.314307,76.378398,...,1250600,0.0,0.0,25.986160,26.121104,25.774107,25.957245,365400,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 00:00:00-05:00,181.270004,182.570007,179.529999,180.750000,136682600,0.0,0.0,137.509995,138.029999,135.860001,...,5541300,0.0,0.0,31.510000,31.820000,31.280001,31.750000,754200,0.0,0.0
2024-03-01 00:00:00-05:00,179.550003,180.529999,177.380005,179.660004,73488000,0.0,0.0,137.029999,139.149994,134.669998,...,2752900,0.0,0.0,31.709999,32.150002,31.650000,32.070000,536300,0.0,0.0
2024-03-04 00:00:00-05:00,176.149994,176.899994,173.789993,175.100006,81510100,0.0,0.0,139.149994,143.490005,138.809998,...,2051900,0.0,0.0,32.040001,32.509998,32.029999,32.060001,681100,0.0,0.0


In [10]:
data.to_csv('dataframe_full.csv')